In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class EncMNIST(nn.Module):
    def __init__(self, latent_dim_mnist):
        super(EncMNIST, self).__init__()
        self.latent_dim_mnist = latent_dim_mnist
        self.dim_MNIST = 28 * 28

        self.enc = nn.Sequential(nn.Linear(self.dim_MNIST, 512),
                                 nn.ReLU(inplace=True), 
                                 nn.Linear(512, 128),
                                 nn.ReLU(inplace=True))
        self.enc_mu_mnist = nn.Linear(128, latent_dim_mnist)
        self.enc_var_mnist = nn.Linear(128, latent_dim_mnist)

    def forward(self, x):
        x = self.enc(x)
        mu_mnist = self.enc_mu_mnist(x)
        log_var_mnist = self.enc_var_mnist(x)
        return mu_mnist, log_var_mnist
    
class DecMNIST(nn.Module):
    def __init__(self, latent_dim):
        super(DecMNIST, self).__init__()  
        self.latent_dim = latent_dim + 10
        self.dim_MNIST   = 28 * 28
        
        self.dec = nn.Sequential(nn.Linear(self.latent_dim, 128), 
                                 nn.ReLU(inplace=True),
                                 nn.Linear(128, 512), 
                                 nn.ReLU(inplace=True),
                                 nn.Linear(512, self.dim_MNIST), 
                                 nn.Sigmoid())
        
    def forward(self, z,x,generate_mode):
#         return self.dec(z).reshape(-1, 1, 28, 28).to(z.device)
        return self.dec(z).to(device)
    
    
class pixelcnn_decoder(nn.Module):
    def __init__(self, pixelcnn):
        super(pixelcnn_decoder, self).__init__()
        self.pixelcnn = pixelcnn

    def forward(self, z, img,generate_mode):
        img_out = img.reshape(z.shape[0],1,28,28).to(z.device)
        if generate_mode is False:
            sample = self.pixelcnn(img_out, z )
        else:
            shape = [1,28,28]
            count = z.shape[0]
            sample = self.pixelcnn.sample(img_out,shape,count, z )
#             sample = self.pixelcnn(img_out, z )

        return (sample, )